# Overview of Notebooks

* [HAPI_01.ipynb - Basics](HAPI_01.ipynb) 
* **[HAPI_02.ipynb - Data structures](HAPI_02.ipynb) (this Notebook)**
* [HAPI_03.ipynb - Plotting](HAPI_03.ipynb)
* [HAPI_04.ipynb - Problems](HAPI_04.ipynb)

# Setup

In [1]:
# Show Matplotlib plots in page instead of opening a window
%matplotlib inline 
# Have Matplotlib create vector (svg) instead of raster (png) images
%config InlineBackend.figure_formats = ['svg'] 

# Misc. configuration
import warnings
# See https://github.com/boto/boto3/issues/454 for an explanation.
warnings.simplefilter("ignore", ResourceWarning) # removes spurious Notebook warns

# Data Model

A request for data of the form

```python
data, meta = hapi(server, dataset, parameters, start, stop)
```

returns the [Numpy N-D array](https://docs.scipy.org/doc/numpy-1.15.1/user/quickstart.html) `data` and a Python dictionary `meta` from a HAPI-compliant data server `server`. The structure of `meta` mirrors the structure of the metadata response from a HAPI server.

The basic data structure returned by all HAPI servers is a CSV file in which the first column is a time stamp and subsequent columns are data measured or associated with that time stamp. The columns are mapped to one or more parameters (that may be multi-dimensional arrays) using the metadata associated with the request for CSV data.

For more information on the HAPI server specification, see https://github.com/hapi-server/data-specification. Note that `hapiclient` requests HAPI Binary from a server if that feature is available; a HAPI compliant server is only required to serve HAPI CSV.

# Extracting Data

In [8]:
from hapiclient import hapi

server     = 'http://hapi-server.org/servers/TestData2.0/hapi'
dataset    = 'dataset1'
parameters = 'scalar,vector'
start      = '1970-01-01T00:00:00'
stop       = '1970-01-01T00:00:10'

data,meta = hapi(server,dataset,parameters,start,stop)

`data` is a Numpy N-D array with named fields `Time`, `scalar`, and `vector`. The array has 10 elements (one for each time value) and each element is a list.

In [9]:
data

array([(b'1970-01-01T00:00:00.000Z', 0.        , [ 0.        , -0.70710678, -1.        ]),
       (b'1970-01-01T00:00:01.000Z', 0.00523596, [ 0.00523596, -0.7033947 , -0.99998629]),
       (b'1970-01-01T00:00:02.000Z', 0.01047178, [ 0.01047178, -0.69966334, -0.99994517]),
       (b'1970-01-01T00:00:03.000Z', 0.01570732, [ 0.01570732, -0.6959128 , -0.99987663]),
       (b'1970-01-01T00:00:04.000Z', 0.02094242, [ 0.02094242, -0.69214317, -0.99978068]),
       (b'1970-01-01T00:00:05.000Z', 0.02617695, [ 0.02617695, -0.68835458, -0.99965732]),
       (b'1970-01-01T00:00:06.000Z', 0.03141076, [ 0.03141076, -0.68454711, -0.99950656]),
       (b'1970-01-01T00:00:07.000Z', 0.03664371, [ 0.03664371, -0.68072087, -0.99932839]),
       (b'1970-01-01T00:00:08.000Z', 0.04187565, [ 0.04187565, -0.67687597, -0.99912283]),
       (b'1970-01-01T00:00:09.000Z', 0.04710645, [ 0.04710645, -0.67301251, -0.99888987])],
      dtype=[('Time', 'S24'), ('scalar', '<f8'), ('vector', '<f8', (3,))])

Access all values for parameter `Time`

In [10]:
data['Time']

array([b'1970-01-01T00:00:00.000Z', b'1970-01-01T00:00:01.000Z',
       b'1970-01-01T00:00:02.000Z', b'1970-01-01T00:00:03.000Z',
       b'1970-01-01T00:00:04.000Z', b'1970-01-01T00:00:05.000Z',
       b'1970-01-01T00:00:06.000Z', b'1970-01-01T00:00:07.000Z',
       b'1970-01-01T00:00:08.000Z', b'1970-01-01T00:00:09.000Z'],
      dtype='|S24')

Convert elements of `Time` to Python `datetime` objects

In [11]:
from hapiclient import hapitime2datetime
dateTime = hapitime2datetime(data['Time'])
dateTime

array([datetime.datetime(1970, 1, 1, 0, 0, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 1, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 2, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 3, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 4, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 5, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 6, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 7, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 8, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 9, tzinfo=<UTC>)], dtype=object)

Access all values for parameter `vector`

In [12]:
data['vector']

array([[ 0.        , -0.70710678, -1.        ],
       [ 0.00523596, -0.7033947 , -0.99998629],
       [ 0.01047178, -0.69966334, -0.99994517],
       [ 0.01570732, -0.6959128 , -0.99987663],
       [ 0.02094242, -0.69214317, -0.99978068],
       [ 0.02617695, -0.68835458, -0.99965732],
       [ 0.03141076, -0.68454711, -0.99950656],
       [ 0.03664371, -0.68072087, -0.99932839],
       [ 0.04187565, -0.67687597, -0.99912283],
       [ 0.04710645, -0.67301251, -0.99888987]])

Access first element (all parameters at first time)

In [13]:
data[0]

(b'1970-01-01T00:00:00.000Z', 0., [ 0.        , -0.70710678, -1.        ])

Access value of `vector` at second timestep.

In [14]:
data['vector'][1] 

array([ 0.00523596, -0.7033947 , -0.99998629])

Access value of second component of `vector` at second timestep

In [15]:
data['vector'][1,1]

-0.7033947028105039

<div style="background-color:yellow">
<h3>Problem 02a</h3>

<p>Find the average radial distance of the moon in March of 2022. (To avoid 100+ users requesting data from the same data server, please use only this month; the data required to solve this problem is locally cached and <code>hapi</code> will use this cached data by default.)</p>
</h3>

In [12]:
from hapiclient import hapi

server     = 'https://hapi-server.org/servers/SSCWeb/hapi'
dataset    = 'moon'
parameters = 'X_GEO'
start      = '2022-01-01T00:00:00.000Z'
stop       = '2022-01-10T00:00:00.000Z'

data, meta = hapi(server, dataset, parameters, start, stop);

# Your code here

hapi(): Running hapi.py version 0.2.3
hapi(): file directory = hapi-server.org_servers_SSCWeb_hapi
hapi(): Reading moon_X_GEO_20220101T000000000_20220110T000000000.pkl
hapi(): Reading moon_X_GEO_20220101T000000000_20220110T000000000.npy 


# Time Representation

A HAPI-compliant server represents time as an ISO 8601 string (with several constraints - see the [HAPI specification](https://github.com/hapi-server/data-specification/blob/master/hapi-dev/HAPI-data-access-spec-dev.md#representation-of-time)). `hapi.py` reads these into a NumPy array of [Python byte literals](https://stackoverflow.com/a/6273618). To convert byte literals to Python `datetime` objects, the function [`hapitime2datetime`](https://github.com/hapi-server/client-python/blob/master/hapiclient/hapi.py) can be used. Internally, this function uses `pandas.to_datetime` for parsing if possible. Otherwise it falls back to a manual method for parsing. The byte literals can also be converted to Astropy time objects as shown below.

In [15]:
from hapiclient import hapi
from hapiclient import hapitime2datetime

server     = 'http://hapi-server.org/servers/TestData2.0/hapi'
dataset    = 'dataset1'
parameters = 'scalar,vector'
start      = '1970-01-01T00:00:00'
stop       = '1970-01-01T00:00:10'

data, meta = hapi(server, dataset, parameters, start, stop)

In [17]:
data['Time']

array([b'1970-01-01T00:00:00.000Z', b'1970-01-01T00:00:01.000Z',
       b'1970-01-01T00:00:02.000Z', b'1970-01-01T00:00:03.000Z',
       b'1970-01-01T00:00:04.000Z', b'1970-01-01T00:00:05.000Z',
       b'1970-01-01T00:00:06.000Z', b'1970-01-01T00:00:07.000Z',
       b'1970-01-01T00:00:08.000Z', b'1970-01-01T00:00:09.000Z'],
      dtype='|S24')

In [16]:
hapitime2datetime(data['Time'])

array([datetime.datetime(1970, 1, 1, 0, 0, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 1, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 2, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 3, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 4, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 5, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 6, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 7, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 8, tzinfo=<UTC>),
       datetime.datetime(1970, 1, 1, 0, 0, 9, tzinfo=<UTC>)], dtype=object)

# Convert to Pandas DataFrame

As HAPI data are NumPy arrays, conversion to Pandas DataFrames uses the existing `pandas.DataFrame` function.

In [18]:
# Get data for use below
from hapiclient import hapi
from hapiclient import hapitime2datetime

server     = 'http://hapi-server.org/servers/TestData2.0/hapi'
dataset    = 'dataset1'
parameters = 'scalar,vector'
start      = '1970-01-01T00:00:00'
stop       = '1970-01-01T00:00:10'

data, meta = hapi(server,dataset,parameters,start,stop)

import pandas

df_Time = pandas.DataFrame(hapitime2datetime(data['Time']))
df_scalar = pandas.DataFrame(data['scalar'])
df_vector = pandas.DataFrame(data['vector'])

# Create DataFrame
df = pandas.DataFrame()

# Combine DataFrame
df = pandas.concat([df_Time, df_scalar, df_vector], axis=1)

# Name columns
df.columns = ['Time', 'scalar','vector_x', 'vector_y', 'vector_z']

# Set Time to be index
df.set_index('Time', inplace=True)

df

,scalar,vector_x,vector_y,vector_z
Time,,,,
1970-01-01 00:00:00+00:00,0.000000,0.000000,-0.707107,-1.000000
1970-01-01 00:00:01+00:00,0.005236,0.005236,-0.703395,-0.999986
1970-01-01 00:00:02+00:00,0.010472,0.010472,-0.699663,-0.999945
1970-01-01 00:00:03+00:00,0.015707,0.015707,-0.695913,-0.999877
1970-01-01 00:00:04+00:00,0.020942,0.020942,-0.692143,-0.999781
1970-01-01 00:00:05+00:00,0.026177,0.026177,-0.688355,-0.999657
1970-01-01 00:00:06+00:00,0.031411,0.031411,-0.684547,-0.999507
1970-01-01 00:00:07+00:00,0.036644,0.036644,-0.680721,-0.999328
1970-01-01 00:00:08+00:00,0.041876,0.041876,-0.676876,-0.999123


<div style="background-color:yellow">
<h3>Problem 02b</h3>

<p>1. Find the mean and standard deviation of each column using <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html"><code>DataFrame</code></a> methods.</p>
<p>2. Find the time that scalar is a maximum using <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html"><code>DataFrame</code></a> methods.</p>
</h3>

# Convert to NDCube

HAPI's NumPy data arrays can be converted to NDCubes (used by AstroPy) using the `ndcube.NDCube` function.

We also set a WCS array, and create appropraite timestamps.

In [1]:
from hapiclient import hapi
from hapiclient import hapitime2datetime

server     = 'http://hapi-server.org/servers/TestData2.0/hapi'
dataset    = 'dataset1'
parameters = 'scalar,vector'
start      = '1970-01-01T00:00:00'
stop       = '1970-01-01T00:00:10'

data, meta = hapi(server, dataset, parameters, start, stop)

dateTimes = hapitime2datetime(data['Time'])

from datetime import timezone
times = [dt.replace(tzinfo=timezone.utc).timestamp() for dt in dateTimes]

import astropy.wcs
import astropy.units as u
import numpy as np

!pip install ndcube
import ndcube

my_wcs = astropy.wcs.WCS({"CTYPE1": "TIME", 
                          "CUNIT1": "s", 
                          "CDELT1": 1, 
                          "CRPIX1": 0, 
                          "CRVAL1": 0, 
                          "NAXIS1": 10})

#extra_coords = [("time", 0, dateTimes)]
# latest ndcube changed the extra_coords option
#cube = ndcube.NDCube(data['scalar'], my_wcs, extra_coords=extra_coords)
cube = ndcube.NDCube(data['scalar'], my_wcs)

from astropy.time import Time
t=Time(dateTimes)
cube.extra_coords.add('time',0,t)

cube

     |████████████████████████████████| 108 kB 26.4 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 70.4 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 60 kB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 402 kB 81.9 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 191 kB 81.3 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for asdf-coordinates-schemas: filename=asdf_coordinates_schemas-0.1.0-py3-none-any.whl size=19013 sha256=18a0bd0b338befce88fabeee349ea304ca59645006004b0cc080b800931d202d
  Stored in directory: /home/jovyan/.cache/pip/wheels/d2/ec/e5/fa0a97eac37491832995f130264c88aa8e1033689e4cb5211f
  Created wheel for asdf-wcs-schemas

NDCube
------
Dimensions: [10.] pix
Physical Types of Axes: [('time', 'time')]
Unit: None
Data Type: float64

# Generating Data (optional)

For testing, it may be useful to create a simulated HAPI data respsonse in Python. A HAPI response of

```
1970-01-01T00:00:00.000Z, 1.,2.,3.
1970-01-01T00:00:02.000Z, 4.,5.,6.
```

where the metadata indicates there is one parameter named `vector` with `size=[3]` and `type=double` could be created by

In [22]:
import numpy as np
data = np.ndarray(shape=(2), dtype=[('Time', 'S24'), ('vector', '<f8', (3,))])

# Populate: method 1
data['Time'] = np.array([b'1970-01-01T00:00:00.000Z', b'1970-01-01T00:00:01.000Z'])
data['vector'] = np.array([[1.0,2.0,3.0],[4.0,5.0,6.0]])

# Populate: method 2
data[0] = (b'1970-01-01T00:00:00.000Z', [1.0,2.0,3.0])
data[1] = (b'1970-01-01T00:00:01.000Z', [4.0,5.0,6.0])

data

array([(b'1970-01-01T00:00:00.000Z', [1., 2., 3.]),
       (b'1970-01-01T00:00:01.000Z', [4., 5., 6.])],
      dtype=[('Time', 'S24'), ('vector', '<f8', (3,))])

A HAPI response of

```
1970-01-01T00:00:00.000Z, 1.,2.,3.,4.,5.,6.,7.,8.,9.
1970-01-01T00:00:02.000Z, 11.,12.,13.,14.,15.,16.,17.,18.,19.
```
where the metadata indicates there is one parameter named `matrix` with `size=[3,3]` and `type=double` could be created by

In [23]:
import numpy as np

# Allocate
data = np.ndarray(shape=(2), dtype=[('Time', 'S24'), ('matrix', '<f8', (3,3,))])

# Populate
data['Time'] = np.array([b'1970-01-01T00:00:00.000Z', b'1970-01-01T00:00:01.000Z'])
data['matrix'] = np.array( [ [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]], [[11.0,12.0,13.0],[14.0,15.0,16.0],[17.0,18.0,19.0]]] )

data

array([(b'1970-01-01T00:00:00.000Z', [[ 1.,  2.,  3.], [ 4.,  5.,  6.], [ 7.,  8.,  9.]]),
       (b'1970-01-01T00:00:01.000Z', [[11., 12., 13.], [14., 15., 16.], [17., 18., 19.]])],
      dtype=[('Time', 'S24'), ('matrix', '<f8', (3, 3))])

Multiple parameters, e.g. a response with both the vector and matrix parameters considered above

```
1970-01-01T00:00:00.000Z, 1.,2.,3.,  1.,2.,3.,4.,5.,6.,7.,8.,9.
1970-01-01T00:00:02.000Z, 4.,5.,6., 11.,12.,13.,14.,15.,16.,17.,18.,19.
```

can be created by populating

In [24]:
data = np.ndarray(shape=(2), dtype=[('Time', 'S24'), ('vector', '<f8', (3,)), ('matrix', '<f8', (3,3,))])
data['Time'] = np.array([b'1970-01-01T00:00:00.000Z', b'1970-01-01T00:00:01.000Z'])
data['vector'] = np.array([[1.0,2.0,3.0],[4.0,5.0,6.0]])
data['matrix'] = np.array( [ [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]], [[11.0,12.0,13.0],[14.0,15.0,16.0],[17.0,18.0,19.0]]] )
data

array([(b'1970-01-01T00:00:00.000Z', [1., 2., 3.], [[ 1.,  2.,  3.], [ 4.,  5.,  6.], [ 7.,  8.,  9.]]),
       (b'1970-01-01T00:00:01.000Z', [4., 5., 6.], [[11., 12., 13.], [14., 15., 16.], [17., 18., 19.]])],
      dtype=[('Time', 'S24'), ('vector', '<f8', (3,)), ('matrix', '<f8', (3, 3))])


# Metadata

The metadata returned by `hapi()` is a straightforward mapping of the JSON metadata from a HAPI server.  Earlier we showed the metadata for a dataset; now we look at (a) creating a list of all HAPI servers and (b) asking any specific HAPI server which datasets it has available.

## Listing all Servers

HAPI has a query function to return all current HAPI servers, which is identical to the 
[Equivalent URL](https://github.com/hapi-server/data-specification/blob/master/servers.txt).

In [20]:
from hapiclient import hapi

servers = hapi() # servers is an array of URLs
display(servers)

['http://hapi-server.org/servers/SSCWeb/hapi',
 'http://datashop.elasticbeanstalk.com/hapi',
 'https://cdaweb.gsfc.nasa.gov/hapi',
 'http://planet.physics.uiowa.edu/das/das2Server/hapi',
 'https://iswa.gsfc.nasa.gov/IswaSystemWebApp/hapi',
 'http://lasp.colorado.edu/lisird/hapi',
 'http://hapi-server.org/servers/TestData2.0/hapi',
 'http://amda.irap.omp.eu/service/hapi',
 'https://vires.services/hapi']

## Listing all Datasets from a Server

For a given server - in this example, CDAWeb - you can fetch the full list of dataset ids it serves. There is an [Equivalent URL: https://cdaweb.gsfc.nasa.gov/hapi/catalog](https://cdaweb.gsfc.nasa.gov/hapi/catalog) using the `catalog` endpoint (defined in the HAPI specification) that you can use as well.  Here, we only display the first five entries for example use, while a call to `display(meta)` will display them all.

In [22]:
from hapiclient import hapi

server = 'https://cdaweb.gsfc.nasa.gov/hapi'
meta = hapi(server)

display(meta['catalog'][0:5])

[{'id': 'A1_K0_MPA'},
 {'id': 'A2_K0_MPA'},
 {'id': 'AC_AT_DEF'},
 {'id': 'AC_H0_MFI'},
 {'id': 'AC_H0_SWE'}]

## Listing all Parameters in a Dataset

Each dataset's metadata is available as a query, without needing to fetch the actual data, using the `info` endpoint. For this example, there is an [Equivalent URL: https://cdaweb.gsfc.nasa.gov/hapi/info?id=AC_H0_MFI](https://cdaweb.gsfc.nasa.gov/hapi/info?id=AC_H0_MFI)

In [25]:
from hapiclient import hapi

server  = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset = 'AC_H0_MFI'
meta = hapi(server,dataset)
display(meta)

{'HAPI': '2.0',
 'status': {'code': 1200, 'message': 'OK'},
 'parameters': [{'name': 'Time',
   'type': 'isotime',
   'units': 'UTC',
   'length': 24,
   'fill': None},
  {'name': 'Magnitude',
   'type': 'double',
   'units': 'nT',
   'fill': '-1.0E31',
   'description': 'B-field magnitude'},
  {'name': 'BGSEc',
   'type': 'double',
   'units': 'nT',
   'fill': '-1.0E31',
   'description': 'Magnetic Field Vector in GSE Cartesian coordinates (16 sec)',
   'size': [3]},
  {'name': 'BGSM',
   'type': 'double',
   'units': 'nT',
   'fill': '-1.0E31',
   'description': 'Magnetic field vector in GSM coordinates (16 sec)',
   'size': [3]},
  {'name': 'dBrms',
   'type': 'double',
   'units': 'nT',
   'fill': '-1.0E31',
   'description': 'RMS of Magnetic Field (16 sec period)'},
  {'name': 'SC_pos_GSE',
   'type': 'double',
   'units': 'km',
   'fill': '-1.0E31',
   'description': 'ACE s/c position, 3 comp. in GSE coord.',
   'size': [3]},
  {'name': 'SC_pos_GSM',
   'type': 'double',
   'unit

## Listing Parameter Metadata

You can likewise examine a subset of the full dataset (rather than all variables returnable) by adding the same 'parameters' call you would use in a query.
[Equivalent URL: https://cdaweb.gsfc.nasa.gov/hapi/info?id=AC_H0_MFI&parameters=Magnitude,BGSEc](https://cdaweb.gsfc.nasa.gov/hapi/info?id=AC_H0_MFI&parameters=Magnitude,BGSEc)

(Note that HAPI allows 'unlisted' non-standard internal keys that prefixed by `x_`.  Similar to the Python `_name` scheme, this is an advanced feature not covered in this tutorial.)

Here we fetch just the `Magnitude` and `BGSEc` for the `AC_H0_MFI` dataset from `CDAWeb`.

In [28]:
from hapiclient import hapi

server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset    = 'AC_H0_MFI'
parameters = 'Magnitude,BGSEc'

meta = hapi(server,dataset,parameters)

display(meta)

{'HAPI': '2.0',
 'status': {'code': 1200, 'message': 'OK'},
 'parameters': [{'name': 'Time',
   'type': 'isotime',
   'units': 'UTC',
   'length': 24,
   'fill': None},
  {'name': 'Magnitude',
   'type': 'double',
   'units': 'nT',
   'fill': '-1.0E31',
   'description': 'B-field magnitude'},
  {'name': 'BGSEc',
   'type': 'double',
   'units': 'nT',
   'fill': '-1.0E31',
   'description': 'Magnetic Field Vector in GSE Cartesian coordinates (16 sec)',
   'size': [3]}],
 'startDate': '1997-09-02T00:00:12Z',
 'stopDate': '2022-01-03T23:59:58Z',
 'resourceURL': 'https://cdaweb.gsfc.nasa.gov/misc/NotesA.html#AC_H0_MFI',
 'contact': 'N. Ness @ Bartol Research Institute',
 'x_server': 'https://cdaweb.gsfc.nasa.gov/hapi',
 'x_dataset': 'AC_H0_MFI'}

----
Next up, [plotting data](HAPI_03.ipynb)
----